In [2]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 46.9 MB 39 kB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
     |████████████████████████████████| 344 kB 5.4 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595740 sha256=1e18dd16d7c3963ffa9e251bf4d1e5202451314063b1f0af81d7e715cd7e3bde
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp
     |████████████████████████████████| 1.2 MB 5.4 MB/s 
     |████████████████████████████████| 754 kB 5.2 MB/s 
     |████████████████████████████████| 895 kB 36.7 MB/s 
     |████████████████████████████████| 3.0 MB 38.4 MB/s 


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-psyn566z
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-psyn566z
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=7007abfdd99c72229d880b90ede6c7a00467d54ad2db36a059c404fc7319d928
  Stored in directory: /tmp/pip-ephem-wheel-cache-gludsmqm/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [5]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [6]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [43]:
##GPU 사용 시
device = torch.device("cuda")

In [8]:
bertmodel, vocab = get_pytorch_kobert_model()

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


In [9]:
import pandas as pd

# train_data load

In [10]:
data = pd.read_csv('/content/drive/MyDrive/공모전/금융/강남구/강남구/10. 강남구_분식.csv')
data1 = pd.read_csv('/content/drive/MyDrive/공모전/금융/강남구/강남구/3. 강남구_치킨.csv')
data2 = pd.read_csv('/content/drive/MyDrive/공모전/금융/강남구/강남구/5. 강남구_중식.csv')
data3 = pd.read_csv('/content/drive/MyDrive/공모전/금융/강남구/강남구/5. 강남구_중국집.csv')

In [11]:
data4 = pd.read_csv('/content/drive/MyDrive/공모전/금융/마포구_리뷰최종/마포구_분식_리뷰최종.csv')
data5 = pd.read_csv('/content/drive/MyDrive/공모전/금융/마포구_리뷰최종/마포구_중식_리뷰최종.csv')
data6 = pd.read_csv('/content/drive/MyDrive/공모전/금융/마포구_리뷰최종/마포구_치킨_리뷰최종.csv')

In [12]:
del data['Unnamed: 0']

In [14]:
review_data = [data, data1, data2, data3, data4, data5, data6]

In [16]:
#긍정 1 부정 0
for lst in review_data:
    lst['label'] = lst['전체평점'].apply(lambda x: 1 if x >= 4 else 0)

In [24]:
new_data = pd.DataFrame({'리뷰 내용':[], 'label':[]})
for lst in review_data:
    tmp = lst[['리뷰 내용', 'label']]
    new_data = pd.concat([new_data, tmp], axis=0)

In [25]:
new_data.head()

,리뷰 내용,label
0,배달시간은 물론이고 국물이 터져 다른 음식에 샜습니다…,0.0
1,추억의맛마리종류별로시켰어요 맛나요,1.0
2,양도 많고 정말 맛있어요 ㅎㅎ,1.0
3,종류도 많고빠르고 맛있어요,1.0
4,믿고 먹는 스쿨푸드! 역시 맛있습니다. 배달도 빠르고 좋습니다. 앞으로도 많이 시켜...,1.0


In [26]:
new_data.dropna(inplace=True)

In [27]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42) # new_df 오탈자 수정

In [28]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model


# modeling

## dataset 구성

In [29]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [45]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 1
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [31]:
train

,리뷰 내용,label
1041,맛빼고 좋네요....,1.0
63336,맛있어요 또먹을거에요ㅎㅎ,1.0
446,어랏 리뷰를안썻네여 ㅎㅎ 맛나게먹었습니다 굿굿,1.0
13785,맛있습니다~ 펩시가 와서 아쉬웠습니다~,1.0
50041,배부르게 한끼뚝딱함,1.0
...,...,...
943,떡볶이에 양배추 들어가고 많이 안달아서 좋아요. 손수제비도 육수 진하고 맛있네요. ...,1.0
46055,꿀맛입돠,1.0
59927,리뷰이벤트감사해요ㅎㅎ,1.0
74862,너무맛있어요!매번먹어도 먹을때마다감동! 양파후레이크가 계속품절이네용!!흑흑ㅠ 항상맛...,1.0


In [32]:
data_train = BERTDataset(train.values, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(test.values, 0, 1, tok, max_len, True, False)

In [33]:
data_train[0]

(array([   2, 1967, 6464, 5439, 4204, 5703,  517,   54,  517,   54,  517,
          54,  517,   54,    3,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1], dtype=int32),
 array(15, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 1)

In [34]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=5)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=5)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


## bert

In [35]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [36]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [37]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [38]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [39]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [40]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [41]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [46]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/5273 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


epoch 1 batch id 1 loss 0.23840686678886414 train acc 0.921875
epoch 1 batch id 201 loss 0.29160284996032715 train acc 0.951181592039801
epoch 1 batch id 401 loss 0.08830372244119644 train acc 0.9509819201995012
epoch 1 batch id 601 loss 0.11813667416572571 train acc 0.9518770798668885
epoch 1 batch id 801 loss 0.20812007784843445 train acc 0.952227684144819
epoch 1 batch id 1001 loss 0.15892645716667175 train acc 0.9520167332667333
epoch 1 batch id 1201 loss 0.05227252095937729 train acc 0.9518109908409659
epoch 1 batch id 1401 loss 0.1786050647497177 train acc 0.9517532119914347
epoch 1 batch id 1601 loss 0.22759175300598145 train acc 0.9521685665209244
epoch 1 batch id 1801 loss 0.21303948760032654 train acc 0.9524049139367018
epoch 1 batch id 2001 loss 0.0873214527964592 train acc 0.9529922538730635
epoch 1 batch id 2201 loss 0.21083256602287292 train acc 0.9533450704225352
epoch 1 batch id 2401 loss 0.04476098716259003 train acc 0.9538343398583924
epoch 1 batch id 2601 loss 0.1111

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1319 [00:00<?, ?it/s]

epoch 1 test acc 0.9526985405610311


In [47]:
torch.save(model, '/content/drive/MyDrive/공모전/금융/yogiyo_review_model.pt')

# infer

In [73]:
model = torch.load('/content/drive/MyDrive/공모전/금융/yogiyo_review_model.pt')

In [48]:
unseen_test = pd.read_csv('/content/drive/MyDrive/공모전/tmp/강남면류_topic_label.csv')

In [49]:
unseen_test.head()

,Unnamed: 0,시,구,업종명,가게명,년,월,전체평점,맛 평점,양 평점,배달 평점,review,주문 내역,token,topic_score,topic_label
0,21633,서울,강남구,한식,팔당충무김밥국수,2021,7,5,5.0,5.0,5.0,비빔국수도 맛있고 만두가 끝내줬습니다~,\n 세트（돼지불고기＋비빔국수）/1(추가 선택(해물김치만두（8개...,"['비빔국수', '만두']",[0.08237638 0.05348643 0.06796765 0.09465154 0...,4
1,21634,서울,강남구,한식,팔당충무김밥국수,2021,7,5,5.0,5.0,5.0,진짜 맛있었어요~다음날 아침까지 맛있게 먹었어요,\n 충무김밥/2\n,"['진짜', '다음', '날', '아침']",[0.08237638 0.05348643 0.06796765 0.09465154 0...,4
2,21635,서울,강남구,한식,팔당충무김밥국수,2021,7,5,5.0,5.0,5.0,ㄹㅇ 존맛탱 두번드삼 또드삼,"\n 세트（돼지불고기＋멸치국수）/1,세트（돼지불고기＋냉모밀）/1...","['존맛', '두번드삼', '또드삼']",[0.08237638 0.05348643 0.06796765 0.09465154 0...,4
3,21636,서울,강남구,한식,팔당충무김밥국수,2021,7,5,5.0,5.0,5.0,잘 먹었어요 냉면이 안매워서 아쉽...,"\n 팔당매운물냉면/1,만두 반반（갈비5＋해물5）/1\n ...",['냉면'],[0.08237638 0.05348643 0.06796765 0.09465154 0...,4
4,21637,서울,강남구,한식,팔당충무김밥국수,2021,7,5,5.0,5.0,5.0,국수가 맛있어서 진짜 자주 시켜 먹는 것 같아요. 아들이 또 이렇게 얘기하며 옆에서...,"\n 돼지불고기 한접시（250g）/1,충무김밥/1,비빔국수/1,...","['국수', '진짜', '자주', '것', '아들', '또', '얘기', '옆', ...",[0.04741782 0.03078807 0.03912381 0.4788597 0...,3


In [50]:
del unseen_test['Unnamed: 0']

In [51]:
dataset2 = unseen_test

In [52]:
unseen_test = unseen_test[['review']]

In [54]:
unseen_test['label1'] = 9

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [76]:
unseen_values = unseen_test.values
test_set = BERTDataset(unseen_values, 0, 1, tok, max_len, True, False)
test_input = torch.utils.data.DataLoader(test_set, batch_size=1, num_workers=5)
test_output = []
for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
  token_ids = token_ids.long().to(device)
  segment_ids = segment_ids.long().to(device)
  valid_length= valid_length
  out = model(token_ids, valid_length, segment_ids)
  test_output.append(out.tolist())

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/6435 [00:00<?, ?it/s]

In [77]:
np.array(test_output)[:,:,0]

array([[-2.45194483],
       [-2.45221567],
       [-2.4522028 ],
       ...,
       [-2.44695163],
       [-2.48593616],
       [-2.44759989]])

In [70]:
dataset2['sentimental_score_negative'] = np.array(test_output)[:,:,0]

In [71]:
dataset2['sentimental_score_positive'] = np.array(test_output)[:,:,1]

In [72]:
dataset2

,시,구,업종명,가게명,년,월,전체평점,맛 평점,양 평점,배달 평점,review,주문 내역,token,topic_score,topic_label,sentimental_score,sentimental_score_negative,sentimental_score_positive
0,서울,강남구,한식,팔당충무김밥국수,2021,7,5,5.0,5.0,5.0,비빔국수도 맛있고 만두가 끝내줬습니다~,\n 세트（돼지불고기＋비빔국수）/1(추가 선택(해물김치만두（8개...,"['비빔국수', '만두']",[0.08237638 0.05348643 0.06796765 0.09465154 0...,4,"[[-2.4519448280334473, 2.8999648094177246]]",-2.451945,2.899965
1,서울,강남구,한식,팔당충무김밥국수,2021,7,5,5.0,5.0,5.0,진짜 맛있었어요~다음날 아침까지 맛있게 먹었어요,\n 충무김밥/2\n,"['진짜', '다음', '날', '아침']",[0.08237638 0.05348643 0.06796765 0.09465154 0...,4,"[[-2.4522156715393066, 2.906737804412842]]",-2.452216,2.906738
2,서울,강남구,한식,팔당충무김밥국수,2021,7,5,5.0,5.0,5.0,ㄹㅇ 존맛탱 두번드삼 또드삼,"\n 세트（돼지불고기＋멸치국수）/1,세트（돼지불고기＋냉모밀）/1...","['존맛', '두번드삼', '또드삼']",[0.08237638 0.05348643 0.06796765 0.09465154 0...,4,"[[-2.452202796936035, 2.9065935611724854]]",-2.452203,2.906594
3,서울,강남구,한식,팔당충무김밥국수,2021,7,5,5.0,5.0,5.0,잘 먹었어요 냉면이 안매워서 아쉽...,"\n 팔당매운물냉면/1,만두 반반（갈비5＋해물5）/1\n ...",['냉면'],[0.08237638 0.05348643 0.06796765 0.09465154 0...,4,"[[-0.8164593577384949, 0.8952871561050415]]",-0.816459,0.895287
4,서울,강남구,한식,팔당충무김밥국수,2021,7,5,5.0,5.0,5.0,국수가 맛있어서 진짜 자주 시켜 먹는 것 같아요. 아들이 또 이렇게 얘기하며 옆에서...,"\n 돼지불고기 한접시（250g）/1,충무김밥/1,비빔국수/1,...","['국수', '진짜', '자주', '것', '아들', '또', '얘기', '옆', ...",[0.04741782 0.03078807 0.03912381 0.4788597 0...,3,"[[-0.7485471367835999, 0.8204862475395203]]",-0.748547,0.820486
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6430,서울,강남구,한식,오장동함흥냉면-대치점,2020,9,5,5.0,5.0,5.0,드디어 비냉집을 찾았네요 자주 시켜먹을께요,\n 서울5대 비빔냉면/1(추가 선택(곱배기 추가))\n ...,"['비냉집', '자주']",[0.08237638 0.05348643 0.06796765 0.09465154 0...,4,"[[-1.311450481414795, 1.6249232292175293]]",-1.311450,1.624923
6431,서울,강남구,한식,오장동함흥냉면-대치점,2020,9,5,5.0,5.0,5.0,"냉면 간이 다른 양념 안 넣어도 딱 좋고, 김치만두도 속이 꽉 차 너무너무 맛있습니...","\n ICE 물냉면/1,왕김치만두 5개/1\n","['냉면', '간이', '다른', '양념', '안', '김치', '만두', '꽉',...",[0.02564856 0.01665344 0.02116228 0.02947053 0...,8,"[[-2.465561866760254, 2.906245231628418]]",-2.465562,2.906245
6432,서울,강남구,한식,오장동함흥냉면-대치점,2020,9,5,5.0,5.0,5.0,가성비면에서 맛과 양 모두 만족 계속 시켜먹을거같음,\n 서울5대 비빔냉면/1(추가 선택(곱배기 추가))\n ...,"['가성', '맛', '양', '모두', '만족', '계속']",[0.04741782 0.4551641 0.03912381 0.0544837 0...,1,"[[-2.4469516277313232, 2.900794744491577]]",-2.446952,2.900795
6433,서울,강남구,한식,오장동함흥냉면-대치점,2020,9,5,4.0,5.0,5.0,새우만듀 맛있어요ㅠㅠ❣️ 비빔냉면도 맛있었는데 개인적으로는 조금 많이 달았어요..!...,\n 서울5대 비빔냉면/1(만두 추가 선택(새우만두 6개 추가)...,"['새우', '듀', '비빔냉면', '개인', '조금', '배달']",[0.04741782 0.03078807 0.03912381 0.0544837 0...,4,"[[-2.485936164855957, 2.930814266204834]]",-2.485936,2.930814


In [75]:
dataset2.to_csv('/content/drive/MyDrive/공모전/금융/sentimental_score_kobert_강남_면류.csv', encoding = 'utf-8')

In [ ]:
# import pickle

In [ ]:
# with open('/content/drive/MyDrive/공모전/tmp/kobert.pkl', 'wb') as f:
#     pickle.dump(model, f)